# LLMS

In [ ]:
from config import *

In [2]:
from langchain.llms import OpenAI
llm = OpenAI()

In [3]:
llm("On what day is Pakistan's Independence Day celebrated?")

"\n\nPakistan's Independence Day is celebrated on August 14th."

In [4]:
llm_results = llm.generate(["Tell me a joke", "Tell me a poem"]*3)

In [14]:
print(llm_results.generations[0][0].text)



Q: What did the fish say when it hit the wall?
A: Dam!


In [15]:
llm_results.llm_output

{'token_usage': {'prompt_tokens': 24,
  'total_tokens': 388,
  'completion_tokens': 364},
 'model_name': 'text-davinci-003'}

# Async API

In [18]:
import time
import asyncio

from langchain.llms import OpenAI

def generate_serially():
    llm = OpenAI(temperature=0.9)
    for _ in range(10):
        resp = llm.generate(["Hello, how are you?"])
        print(resp.generations[0][0].text)
        
async def async_generate(llm):
    resp = await llm.agenerate(["Hello, how are you?"])
    print(resp.generations[0][0].text)
    
async def generate_concurrently():
    llm = OpenAI(temperature=0.9)
    tasks = [async_generate(llm) for _ in range(10)]
    await asyncio.gather(*tasks)
    
s = time.perf_counter()

await generate_concurrently()

elapsed = time.perf_counter() - s

print("\033[1m" + f"Concurrent executed in {elapsed:0.2f} seconds." + "\033[0m")

s = time.perf_counter()

generate_serially()
elapsed = time.perf_counter() - s

print("\033[1m" + f"Serial executed in {elapsed:0.2f} seconds." + "\033[0m")



I'm doing great, thank you. How about yourself?


I'm doing well, thank you. How about you?


I'm doing well. How about you?


I'm doing well, thank you. How about you?


I'm doing well, thanks. How about yourself?


I'm doing well, thank you. How about you?


I'm doing well, thank you. How about yourself?


I'm doing well, thanks for asking. How about you?


I'm doing well, thank you for asking. How about you?


I'm doing well, thank you. How about you?
Concurrent executed in 1.30 seconds.


I'm doing well, thank you. How about you?


I'm doing well, thank you. How about you?


I'm doing well, thank you. How about you?


I'm doing well, thank you. How about you?


I'm doing well, thank you. How about you?


I'm doing well, thank you. How about you?


I'm doing well, thank you for asking. How about you?


I'm doing well, thank you. How about you?


I'm doing well, thank you. How about you?


I'm doing alright, and yourself?
Concurrent executed in 8.48 seconds.


# Custom LLM

In [1]:
from typing import Any, List, Mapping, Optional

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM

In [2]:
class CustomLLM(LLM):
    n: int
    
    @property
    def _llm_type(self) -> str:
        return "CustomLLM"
    
    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
    ) -> str:
        
        if stop is not None:
            raise ValueError("stop is not supported for CustomLLM")
        return prompt[: self.n]

In [3]:
llm = CustomLLM(n=10)

In [4]:
llm("This is a foobar thing")

'This is a '

In [5]:
print(llm)

CustomLLM
Params: {}


# Fake LLM

In [6]:
from langchain.llms.fake import FakeListLLM

In [7]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

In [9]:
tools = load_tools(["python_repl"])

In [10]:
tools

[PythonREPLTool(name='Python_REPL', description='A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.', args_schema=None, return_direct=False, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, handle_tool_error=False, python_repl=PythonREPL(globals={'__name__': 'langchain.tools.python.tool', '__doc__': 'A tool for running python code in a REPL.', '__package__': 'langchain.tools.python', '__loader__': <_frozen_importlib_external.SourceFileLoader object at 0x7f44265a98a0>, '__spec__': ModuleSpec(name='langchain.tools.python.tool', loader=<_frozen_importlib_external.SourceFileLoader object at 0x7f44265a98a0>, origin='/home/concaption/Git/LangChainPractice/.venv/lib/python3.10/site-packages/langchain/tools/python/tool.py'), '__file__': '/home/concaption/Git/LangChainPractice/.venv/lib/python3.10/site-packages/langchain/tools/python/tool.

In [20]:
responses = ["Action: Python_REPL\nAction Input: print(2+2)", "Final Answer: 4"]
llm = FakeListLLM(responses=responses)

In [21]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [23]:
agent.run("whats 2 + 4")



> Entering new AgentExecutor chain...
Action: Python_REPL
Action Input: print(2+2)
Observation: 4

Thought:Final Answer: 4

> Finished chain.


'4'

# Human Input LLM

In [24]:
from langchain.llms.human import HumanInputLLM

In [25]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

In [29]:
tools = load_tools(['wikipedia'])
llm = HumanInputLLM(
    prompt_func=lambda prompt: print(
        f"\n===PROMPT===\n{prompt}\n===END PROMPT===\n"
    )
)

In [33]:
agent = initialize_agent(
    tools=tools,llm=llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [35]:
agent.run("What is 'Boochi the Rock!'")



> Entering new AgentExecutor chain...

===PROMPT===
Answer the following questions as best you can. You have access to the following tools:

Wikipedia: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Wikipedia]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: What is 'Boochi the Rock!'
Thought:
===END PROMPT===



OutputParserException: Could not parse LLM output: `I need to use a tool
use wikipedia
wikipedia
Action: wikipedia
    Page: Manga Time Kirara
    Page: Manga Time Kirara
Wi
aadasf`

# Caching

In [37]:
import langchain
from langchain.llms import OpenAI
llm = OpenAI(model_name='text-davinci-002', n=2, best_of=2)

### In memeory cache

In [38]:
from langchain.cache import InMemoryCache
langchain.llm_cache = InMemoryCache()

llm.predict("tell me a joke")


'\n\nWhy did the chicken cross the road?\n\nTo get to the other side.'

In [39]:
llm.predict("tell me a joke")

'\n\nWhy did the chicken cross the road?\n\nTo get to the other side.'

### SQLite Cache

In [40]:
from langchain.cache import SQLiteCache

langchain.llm_cache = SQLiteCache(database_path=".langchain.db")

In [43]:
llm.predict("tell me a joke")

'\n\nWhy did the chicken cross the road?\n\nTo get to the other side.'

### Optional Caching in Chains

In [45]:
llm = OpenAI(model_name="text-davinci-002")
no_cache_llm = OpenAI(model_name="text-davinci-002", cache=False)

In [46]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain

text_splitter = CharacterTextSplitter()

In [48]:
with open('loremipsum.txt') as f:
    lorem_ipsum = f.read()
texts = text_splitter.split_text(lorem_ipsum)

In [49]:
from langchain.docstore.document import Document

docs = [Document(page_content=t) for t in texts[:3]]
from langchain.chains.summarize import load_summarize_chain

In [50]:
chain = load_summarize_chain(llm, chain_type="map_reduce", reduce_llm=no_cache_llm)

In [51]:
chain.run(docs)

'\n\nLorem ipsum dolor sit amet, consectetur adipiscing elit. Donec id mauris vel justo feugiat luctus. Proin eget condimentum ipsum. Nunc tempus posuere lacus non mattis. Sed maximus venenatis nulla, sit amet condimentum tortor pulvinar nec. Integer id dui vel ligula blandit venenatis. In eu porta velit. Aenean commodo aliquam erat, ac hendrerit enim porta et. Nunc lorem augue, scelerisque quis mollis a, tristique sit amet sem.\n\nFusce consequat eros vitae luctus luctus. Suspendisse potenti. Nunc sodales id ligula sit amet dapibus. Etiam iaculis scelerisque lorem a tempus. Fusce commodo rutrum neque, ut facilisis ex. Suspendisse potenti. Vivamus mollis, tur'

# Serialization

In [52]:
from langchain.llms import OpenAI
from langchain.llms.loading import load_llm

### Loading

In [57]:
cat llm.json

{
    "model_name": "text-davinci-003",
    "temperature": 0.7,
    "max_tokens": 256,
    "top_p": 1.0,
    "frequency_penalty": 0.0,
    "presence_penalty": 0.0,
    "n": 1,
    "best_of": 1,
    "request_timeout": null,
    "_type": "openai"
}

In [58]:
llm = load_llm("llm.json")

In [59]:
llm = load_llm("llm.yaml")

### Saving

In [60]:
llm.save("llm.json")

# Streaming

In [61]:
from langchain.llms import OpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm = OpenAI(streaming=True, callbacks=[StreamingStdOutCallbackHandler()], temperature=0)

In [62]:
resp = llm("Write me a song about sparkling water")



Verse 1
I'm sippin' on sparkling water,
It's so refreshing and light.
It's the perfect way to quench my thirst
On a hot summer night.

Chorus
Sparkling water, sparkling water,
It's the best way to stay hydrated.
It's so crisp and so clean,
It's the perfect way to stay lean.

Verse 2
I'm sippin' on sparkling water,
It's so bubbly and bright.
It's the perfect way to cool me down
On a hot summer night.

Chorus
Sparkling water, sparkling water,
It's the best way to stay hydrated.
It's so crisp and so clean,
It's the perfect way to stay lean.

Verse 3
I'm sippin' on sparkling water,
It's so light and so clear.
It's the perfect way to keep me cool
On a hot summer night.

Chorus
Sparkling water, sparkling water,
It's the best way to stay hydrated.
It's so crisp and so clean,
It's the perfect way to stay lean.

In [63]:
llm.generate(['tell me a joke'])



Q: What did the fish say when it hit the wall?
A: Dam!

LLMResult(generations=[[Generation(text='\n\nQ: What did the fish say when it hit the wall?\nA: Dam!', generation_info={'finish_reason': 'stop', 'logprobs': None})]], llm_output={'token_usage': {}, 'model_name': 'text-davinci-003'}, run=[RunInfo(run_id=UUID('8407166e-0195-42e3-a6f6-ec193f78de14'))])

# Tracking toke usage

In [64]:
from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback

In [65]:
llm = OpenAI(model_name='text-davinci-002', n=2, best_of=2)

In [71]:
with get_openai_callback() as cb:
    result = llm("tell me a joke .")
    print(cb)

Tokens Used: 0
	Prompt Tokens: 0
	Completion Tokens: 0
Successful Requests: 0
Total Cost (USD): $0.0


In [74]:
with get_openai_callback() as cb:
    result1 = llm("tell me a joke1")
    result2 = llm("tell me a joke1")
    print(cb)

Tokens Used: 0
	Prompt Tokens: 0
	Completion Tokens: 0
Successful Requests: 0
Total Cost (USD): $0.0


In [84]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)
tools = load_tools(["serpapi","llm-math"], llm=llm)
agent = initialize_agent(
    tools=tools, llm=llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [85]:
with get_openai_callback() as cb:
    response = agent.run(
        "Who is Olivia Wilde's boyfriend? What is his current age raised to the 0.23 power?"
    )
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")



> Entering new AgentExecutor chain...
 I need to find out who Olivia Wilde's boyfriend is and then calculate his age raised to the 0.23 power.
Action: Search
Action Input: "Olivia Wilde boyfriend"
Observation: Olivia Wilde started dating Harry Styles after ending her years-long engagement to Jason Sudeikis — see their relationship timeline.
Thought: I need to find out Harry Styles' age.
Action: Search
Action Input: "Harry Styles age"
Observation: 29 years
Thought: I need to calculate 29 raised to the 0.23 power.
Action: Calculator
Action Input: 29^0.23
Observation: Answer: 2.169459462491557
Thought: I now know the final answer.
Final Answer: Harry Styles, Olivia Wilde's boyfriend, is 29 years old and his age raised to the 0.23 power is 2.169459462491557.

> Finished chain.
Total Tokens: 0
Prompt Tokens: 0
Completion Tokens: 0
Total Cost (USD): $0.0


# Chat Models

In [86]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI()

In [88]:
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage,
)

human_message = HumanMessage(content="Translate this sentence from English to French:  I love programming.")
chat([human_message])

AIMessage(content="J'adore programmer.", additional_kwargs={}, example=False)

In [89]:
messages = [
    SystemMessage(content="You are a helpful assistant that translates English to French."),
    HumanMessage(content="Hello, how are you?"),
]

In [90]:
chat(messages=messages)

AIMessage(content='Bonjour, comment ça va ?', additional_kwargs={}, example=False)

In [91]:
batch_messages = [
    [
        SystemMessage(content="You are a helpful assistant that translates English to French"),
        HumanMessage(content="I love programming."),
    ],
    [
        SystemMessage(content="You are a helpful assistant that translates English to French"),
        HumanMessage(content="I love artifical intelligence."),
    ]
]

In [92]:
result = chat.generate(batch_messages)

In [93]:
result

LLMResult(generations=[[ChatGeneration(text="J'adore la programmation.", generation_info={'finish_reason': 'stop'}, message=AIMessage(content="J'adore la programmation.", additional_kwargs={}, example=False))], [ChatGeneration(text="J'adore l'intelligence artificielle.", generation_info={'finish_reason': 'stop'}, message=AIMessage(content="J'adore l'intelligence artificielle.", additional_kwargs={}, example=False))]], llm_output={'token_usage': {'prompt_tokens': 53, 'completion_tokens': 20, 'total_tokens': 73}, 'model_name': 'gpt-3.5-turbo'}, run=[RunInfo(run_id=UUID('de7fc965-3b2f-4619-b636-2812069ff69e')), RunInfo(run_id=UUID('e92d44fc-ce97-4dfd-9c59-65c69e359c33'))])

In [94]:
result.llm_output

{'token_usage': {'prompt_tokens': 53,
  'completion_tokens': 20,
  'total_tokens': 73},
 'model_name': 'gpt-3.5-turbo'}

# Caching

In [95]:
import langchain
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI()

### In Memory Cache

In [99]:
from langchain.cache import InMemoryCache
langchain.llm_cache = InMemoryCache()

llm.predict('tell me joke')

"Sure, here's a classic one:\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything!"

In [100]:
llm.predict("tell me joke")

"Sure, here's a classic one:\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything!"

### SQLite Cache

In [101]:
import langchain
from langchain.cache import SQLiteCache

langchain.llm_cache = SQLiteCache('.langchain.db')

In [102]:
llm.predict("I am a student")

"That's great! As a student, you have the opportunity to learn and grow academically and personally. What level of education are you currently pursuing, and what subjects or areas are you interested in?"

In [103]:
llm.predict("I am a student")

"That's great! As a student, you have the opportunity to learn and grow academically and personally. What level of education are you currently pursuing, and what subjects or areas are you interested in?"

# Human input chat model

In [104]:
from langchain.chat_models.human import HumanInputChatModel

In [105]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

In [106]:
tools = load_tools(['wikipedia'])
llm = HumanInputChatModel()

In [107]:
agent = initialize_agent(
    tools=tools, llm=llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [108]:
agent.run("What is Bocchi the Rock?")



> Entering new AgentExecutor chain...

 ======= start of message ======= 


type: system
data:
  content: "Answer the following questions as best you can. You have access to the following tools:\n\nWikipedia: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\n\nThe way you use the tools is by specifying a json blob.\nSpecifically, this json should have a `action` key (with the name of the tool to use) and a `action_input` key (with the input to the tool going here).\n\nThe only values that should be in the \"action\" field are: Wikipedia\n\nThe $JSON_BLOB should only contain a SINGLE action, do NOT return a list of multiple actions. Here is an example of a valid $JSON_BLOB:\n\n```\n{\n  \"action\": $TOOL_NAME,\n  \"action_input\": $INPUT\n}\n```\n\nALWAYS use the following format:\n\nQuestion: the input question you must answer\nThought: you sho

KeyboardInterrupt: Interrupted by user

#  LLMChain

In [111]:
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate
from langchain.schema import HumanMessage

In [135]:
message = HumanMessage(content="What is your name? {text}")

chat_prompt = ChatPromptTemplate.from_messages([message])

In [136]:
chain = LLMChain(llm=chat, prompt=chat_prompt)

In [145]:
chain.run(text="How are you")

'I am an AI language model developed by OpenAI and I am often referred to as GPT-3.'

# Prompts

In [1]:
from langchain import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

In [2]:
template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_message = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_message)

In [6]:
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

chat_prompt.format_prompt(input_language="English", output_language="French", text="I love programming.").to_messages()

[SystemMessage(content='You are a helpful assistant that translates English to French.', additional_kwargs={}),
 HumanMessage(content='I love programming.', additional_kwargs={}, example=False)]

In [7]:
prompt = PromptTemplate(
    template="You are a helpful assistant that translates {input_language} to {output_language}.",
    input_variables=["input_language", "output_language"],
)

system_message_prompt = SystemMessagePromptTemplate(prompt=prompt)

# Streaming

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    HumanMessage,
)
from config import *

In [11]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
chat = ChatOpenAI(streaming=True, callbacks=[StreamingStdOutCallbackHandler()], temperature=0)
resp = chat([HumanMessage(content="Write me a song about sparkling water.")])

(Verse 1)
In a world of thirst, where the sun shines bright,
There's a drink that brings delight, day or night,
It's a bubbly sensation, a refreshing sensation,
Let me sing to you about sparkling water's creation.

(Chorus)
Oh, sparkling water, you're a sparkling delight,
Bubbles dancing in my glass, sparkling so bright,
With every sip, you quench my thirst,
Sparkling water, you're the one I choose first.

(Verse 2)
From mountain springs, you're born pure and clear,
Effervescent bubbles, bringing joy near,
A symphony of fizz, a melody so sweet,
Sparkling water, you make my taste buds meet.

(Chorus)
Oh, sparkling water, you're a sparkling delight,
Bubbles dancing in my glass, sparkling so bright,
With every sip, you quench my thirst,
Sparkling water, you're the one I choose first.

(Bridge)
When life gets heavy, and troubles arise,
Sparkling water, you're my paradise,
You lift my spirits, like a sparkling dream,
With your effervescence, you make life gleam.

(Verse 3)
No need for sugar